In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# %matplotlib widget
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import xarray as xr
import numpy as np
import pandas as pd

In [4]:
xr.set_options(keep_attrs=True)

In [5]:
import ecephys_analyses as ea 
from ecephys import utils
from ecephys.signal.xarray_utils import filter_dataset_by_hypnogram

In [6]:
subjects = ["Segundo", "Valentino", "Doppio", "Alessandro", "Allan"]
experiment = "sleep-homeostasis"

norm_condition = "light-period-circadian-match"
light_period_start = '09:00:00'
light_period_end = '21:00:00'
norm_states = ["N1", "N2"]

condition = "recovery-sleep"
states = ["N1", "N2"]
duration = '02:00:00'

In [14]:
def get_power_summary(f_range, band_name, normalize=True):
    mean_power = dict()
    total_power = dict()

    for subject in subjects:
        if normalize:
            norm_hyp = ea.load_hypnogram(subject, experiment, norm_condition)
            norm_hyp = norm_hyp.keep_states(norm_states).keep_between(light_period_start, light_period_end)

            norm_pwr = ea.load_power(subject, experiment, norm_condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim="frequency")
            norm_pwr = filter_dataset_by_hypnogram(norm_pwr, norm_hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])

        hyp = ea.load_hypnogram(subject, experiment, condition)
        hyp = hyp.keep_states(states).keep_first(duration)

        pwr = ea.load_power(subject, experiment, condition, "spg.nc").sel(frequency=slice(*f_range)).sum(dim='frequency')
        pwr = filter_dataset_by_hypnogram(pwr, hyp).swap_dims({'time': 'datetime'}).drop_vars(['time', 'timedelta'])
        if normalize:
            pwr = pwr / norm_pwr.mean(dim="datetime") * 100

        mean_power[subject] = pwr.to_dataframe().mean()
        total_power[subject] = pwr.to_dataframe().sum()

    mean_power_df = pd.concat(mean_power, names=['subject', 'region']).to_frame(name=f'mean_{band_name}')
    total_power_df = pd.concat(total_power, names=['subject', 'region']).to_frame(name=f'total_{band_name}')
    df = mean_power_df.join(total_power_df)
    
    return df

In [8]:
low_delta_df = get_power_summary((0, 1), 'low_delta')
delta_df = get_power_summary((0.5, 4), 'delta')
theta_df = get_power_summary((5, 10), 'theta')
fast_gamma_df = get_power_summary((60, 120), 'fast_gamma')
df = pd.concat([low_delta_df, delta_df, theta_df, fast_gamma_df], axis=1).assign(condition='recovery-sleep-first2h')
df.to_csv('recovery-sleep-first2h-bandpower.csv')

In [15]:
low_delta_df = get_power_summary((0, 1), 'low_delta', False)
delta_df = get_power_summary((0.5, 4), 'delta', False)
theta_df = get_power_summary((5, 10), 'theta', False)
fast_gamma_df = get_power_summary((60, 120), 'fast_gamma', False)
df = pd.concat([low_delta_df, delta_df, theta_df, fast_gamma_df], axis=1).assign(condition='recovery-sleep-first2h')
df.to_csv('recovery-sleep-first2h-bandpower-unnormalized.csv')